In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 4}
----------
[[ 1266  5119]
 [    0 16906]]
----------
              precision    recall  f1-score   support

           0       1.00      0.20      0.33      6385
           1       0.77      1.00      0.87     16906

    accuracy                           0.78     23291
   macro avg       0.88      0.60      0.60     23291
weighted avg       0.83      0.78      0.72     23291

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.25032985, 0.27127278, 0.27426469, 0.35454798, 0.33709872,
        0.44032097, 0.44680476, 0.48819482, 0.49268031, 0.51013625,
        0.61286128, 0.67968214, 0.71858048, 0.80285239, 0.85620868,
        0.35106111, 0.37748742, 0.41090178, 0.39095509, 0.43633366,
        0.53157854, 0.62133956, 0.65026128, 0.67120481, 0.68167651,
        0.72456264, 0.77542651, 0.77941537, 0.70810652, 0.71209466]),
 'std_fit_time': array([0.0019964 , 0.00099885, 0.00199854, 0.0194509 , 0.00498617,
        0.00847816, 0.00598288, 0.00448859, 0.0259304 , 0.02842414,
        0.09624231, 0.03540599, 0.03041911, 0.00299215, 0.00349104,
        0.02892256, 0.01645494, 0.03590369, 0.00099719, 0.00648308,
        0.00398874, 0.02593112, 0.0249325 , 0.02493334, 0.00249255,
        0.0014956 , 0.00847733, 0.00049901, 0.02792478, 0.00598371]),
 'mean_score_time': array([0.01496077, 0.0119679 , 0.01396215, 0.01346505, 0.02543163,
        0.01496017, 0.01496017, 0.0144608 , 0.01645875, 0.01